#### Fighting With Unstable Gradients Problem

In [1]:
import tensorflow as tf

In [2]:
class LNSimpleRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.state_size = units
        self.output_size = units
        self.simple_rnn_cell = tf.keras.layers.SimpleRNNCell(units, activation=None) # no activation for LN
        
        self.layer_norm = tf.keras.layers.LayerNormalization()
        self.activation = tf.keras.activations.get(activation)
        
    def call(self, inputs, states):
        outputs, new_states = self.simple_rnn_cell(inputs, states)
        norm_outputs = self.activation(self.layer_norm(outputs))
        return norm_outputs, [norm_outputs]

In [6]:
tf.random.set_seed(42)

custom_ln_model = tf.keras.Sequential([
    tf.keras.layers.RNN(LNSimpleRNNCell(32), return_sequences=True, input_shape=[None,5]),
    tf.keras.layers.Dense(14)
])

In [7]:
def fit_and_evaluate(model, train_set, valid_set, learning_rate, epochs=500):
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_mae", patience=50, restore_best_weights=True)
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    
    history = model.fit(train_set, validation_data=valid_set, epochs=epochs, callbacks=[early_stopping_cb])
    valid_loss, valid_mae = model.evaluate(valid_set)
    return valid_mae * 1e6

In [8]:
import pandas as pd
from pathlib import Path

path = Path("datasets/ridership/CTA_-_Ridership_-_Daily_Boarding_Totals.csv")
df = pd.read_csv(path, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"]  # shorter names
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1)  # no need for total, it's just bus + rail
df = df.drop_duplicates()  # remove duplicated months (2011-10 and 2014-07)

df_mulvar = df[["bus", "rail"]] /1e6
df_mulvar["next_day_type"] = df["day_type"].shift(-1)
df_mulvar = pd.get_dummies(df_mulvar) # one hot encode the day type

mulvar_train = df_mulvar["2016-01":"2018-12"] / 1e6
mulvar_valid = df_mulvar["2019-01":"2019-05"] / 1e6
mulvar_test = df_mulvar["2019-06":] / 1e6

In [11]:
def to_windows(dataset, window_size):
    dataset = dataset.window(window_size, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size))
    return dataset

def to_seq2seq_dataset(series, seq_length=56, ahead=14, target_col=1, batch_size=32, shuffle=False, seed=None):
    ds = to_windows(tf.data.Dataset.from_tensor_slices(series), ahead + 1)
    ds = to_windows(ds, seq_length).map(lambda S: (S[:, 0], S[:, 1:, 1]))
    if shuffle:
        ds = ds.shuffle(8 * batch_size, seed=seed)
    return ds.batch(batch_size)

seq2seq_train = to_seq2seq_dataset(mulvar_train, shuffle=True, seed=42)
seq2seq_valid = to_seq2seq_dataset(mulvar_valid)

In [12]:
fit_and_evaluate(custom_ln_model, seq2seq_train, seq2seq_valid, learning_rate=0.1, epochs=5)

Epoch 1/5
33/33 [==============================] - 4s 47ms/step - loss: 0.0715 - mae: 0.2534 - val_loss: 0.0124 - val_mae: 0.1287
Epoch 2/5
33/33 [==============================] - 1s 29ms/step - loss: 0.0021 - mae: 0.0436 - val_loss: 8.5845e-05 - val_mae: 0.0095
Epoch 3/5
33/33 [==============================] - 1s 33ms/step - loss: 5.7613e-05 - mae: 0.0074 - val_loss: 1.9748e-06 - val_mae: 0.0017
Epoch 4/5
33/33 [==============================] - 1s 34ms/step - loss: 2.0168e-06 - mae: 0.0014 - val_loss: 3.4719e-07 - val_mae: 7.0183e-04
Epoch 5/5
3/3 [==============================] - 0s 20ms/step - loss: 1.0099e-08 - mae: 1.1520e-04


115.19733379827812

#### Creating a Custom RNN Class

In [14]:
class MyRNN(tf.keras.layers.Layer):
    def __init__(self, cell, return_sequences=False, **kwargs):
        super().__init__(**kwargs)
        self.cell = cell
        self.return_sequences = return_sequences

    def get_initial_state(self, inputs):
        try:
            return self.cell.get_initial_state(inputs)
        except AttributeError:
            batch_size = tf.shape(inputs)[0]
            return [tf.zeros([batch_size, self.cell.state_size],
                             dtype=inputs.dtype)]

    @tf.function
    def call(self, inputs):
        states = self.get_initial_state(inputs)
        shape = tf.shape(inputs)
        batch_size = shape[0]
        n_steps = shape[1]
        sequences = tf.TensorArray(
            inputs.dtype, size=(n_steps if self.return_sequences else 0))
        outputs = tf.zeros(shape=[batch_size, self.cell.output_size],
                           dtype=inputs.dtype)
        for step in tf.range(n_steps):
            outputs, states = self.cell(inputs[:, step], states)
            if self.return_sequences:
                sequences = sequences.write(step, outputs)

        if self.return_sequences:
            return tf.transpose(sequences.stack(), [1, 0, 2])
        else:
            return outputs

In [15]:
tf.random.set_seed(42)

custom_model = tf.keras.Sequential([
    MyRNN(LNSimpleRNNCell(32), return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])

In [16]:
fit_and_evaluate(custom_model, seq2seq_train, seq2seq_valid, learning_rate=0.1, epochs=5)

Epoch 1/5
33/33 [==============================] - 3s 40ms/step - loss: 0.0844 - mae: 0.2548 - val_loss: 0.0063 - val_mae: 0.0903
Epoch 2/5
33/33 [==============================] - 1s 33ms/step - loss: 0.0024 - mae: 0.0420 - val_loss: 1.7635e-04 - val_mae: 0.0100
Epoch 3/5
33/33 [==============================] - 1s 34ms/step - loss: 7.6954e-05 - mae: 0.0073 - val_loss: 1.0065e-05 - val_mae: 0.0026
Epoch 4/5
33/33 [==============================] - 1s 34ms/step - loss: 2.4737e-06 - mae: 0.0013 - val_loss: 3.0849e-07 - val_mae: 6.4056e-04
Epoch 5/5
3/3 [==============================] - 0s 25ms/step - loss: 5.1290e-09 - mae: 8.4367e-05


84.36736243311316

#### Tackling The Short-Term Memory Problem: LSTM

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, input_shape=[None,5]),
    tf.keras.layers.Dense(14)
])

fit_and_evaluate(model, seq2seq_train, seq2seq_valid, learning_rate=0.01, epochs=5)

Epoch 1/5
33/33 [==============================] - 4s 45ms/step - loss: 1.6324e-13 - mae: 5.2107e-07 - val_loss: 8.4022e-14 - val_mae: 3.6271e-07
Epoch 2/5
33/33 [==============================] - 1s 24ms/step - loss: 7.1829e-14 - mae: 3.2972e-07 - val_loss: 3.8977e-14 - val_mae: 2.4278e-07
Epoch 3/5
33/33 [==============================] - 1s 30ms/step - loss: 3.8982e-14 - mae: 2.4071e-07 - val_loss: 2.6372e-14 - val_mae: 2.0274e-07
Epoch 4/5
33/33 [==============================] - 1s 28ms/step - loss: 2.7873e-14 - mae: 2.0575e-07 - val_loss: 2.2483e-14 - val_mae: 1.8329e-07
Epoch 5/5
3/3 [==============================] - 0s 15ms/step - loss: 2.1264e-14 - mae: 1.7188e-07


0.17188057199746254

#### GRU

In [17]:
tf.random.set_seed(42)
gru_model = tf.keras.Sequential([
    tf.keras.layers.GRU(32, return_sequences=True, input_shape=[None, 5]),
    tf.keras.layers.Dense(14)
])

In [18]:
fit_and_evaluate(gru_model, seq2seq_train, seq2seq_valid, learning_rate=0.1, epochs=5)

Epoch 1/5
33/33 [==============================] - 4s 49ms/step - loss: 6.4876e-14 - mae: 2.7529e-07 - val_loss: 2.4345e-14 - val_mae: 1.6343e-07
Epoch 2/5
33/33 [==============================] - 1s 26ms/step - loss: 1.7842e-14 - mae: 1.5804e-07 - val_loss: 1.8388e-14 - val_mae: 1.5072e-07
Epoch 3/5
33/33 [==============================] - 1s 29ms/step - loss: 1.6963e-14 - mae: 1.5778e-07 - val_loss: 1.8562e-14 - val_mae: 1.4817e-07
Epoch 4/5
33/33 [==============================] - 1s 27ms/step - loss: 1.6923e-14 - mae: 1.5670e-07 - val_loss: 1.8357e-14 - val_mae: 1.4944e-07
Epoch 5/5
3/3 [==============================] - 0s 23ms/step - loss: 1.8321e-14 - mae: 1.4966e-07


0.14965523575938278

#### Using One-Dimensional Convolutional Layers to Process Sequences

In [19]:
conv_rnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=4, strides=2, activation="relu", input_shape=[None,5]),
    tf.keras.layers.GRU(32, return_sequences=True),
    tf.keras.layers.Dense(14)
])

In [20]:
longer_train = to_seq2seq_dataset(mulvar_train, seq_length=112, shuffle=True, seed=42)
longer_valid = to_seq2seq_dataset(mulvar_valid, seq_length=112)
downsampled_train = longer_train.map(lambda X, Y: (X, Y[:, 3::2]))
downsampled_valid = longer_valid.map(lambda X, Y: (X, Y[:, 3::2]))

In [21]:
fit_and_evaluate(conv_rnn_model, downsampled_train, downsampled_valid, learning_rate=0.1, epochs=5)

Epoch 1/5
31/31 [==============================] - 4s 50ms/step - loss: 6.9585e-14 - mae: 2.8881e-07 - val_loss: 2.4840e-14 - val_mae: 1.6690e-07
Epoch 2/5
31/31 [==============================] - 1s 33ms/step - loss: 1.8317e-14 - mae: 1.6285e-07 - val_loss: 1.9120e-14 - val_mae: 1.4948e-07
Epoch 3/5
31/31 [==============================] - 1s 32ms/step - loss: 1.7069e-14 - mae: 1.5558e-07 - val_loss: 1.8397e-14 - val_mae: 1.4994e-07
Epoch 4/5
31/31 [==============================] - 1s 38ms/step - loss: 1.6903e-14 - mae: 1.5556e-07 - val_loss: 1.8295e-14 - val_mae: 1.4893e-07
Epoch 5/5
1/1 [==============================] - 0s 120ms/step - loss: 1.8189e-14 - mae: 1.4830e-07


0.14830187922143523

#### WaveNet

In [23]:
wavenet_model = tf.keras.Sequential()
wavenet_model.add(tf.keras.layers.Input(shape=[None,5]))

for rate in (1,2,4,8) * 2:
    wavenet_model.add(
        tf.keras.layers.Conv1D(filters=32, kernel_size=2, padding="causal", activation="relu", dilation_rate=rate)
    )
wavenet_model.add(tf.keras.layers.Conv1D(filters=14, kernel_size=1))

In [25]:
fit_and_evaluate(wavenet_model, longer_train, longer_valid, learning_rate=0.1, epochs=5)

Epoch 1/5
31/31 [==============================] - 4s 46ms/step - loss: 8.2875e-14 - mae: 3.3152e-07 - val_loss: 2.5451e-14 - val_mae: 1.4973e-07
Epoch 2/5
31/31 [==============================] - 1s 27ms/step - loss: 1.7742e-14 - mae: 1.4713e-07 - val_loss: 1.9287e-14 - val_mae: 1.5273e-07
Epoch 3/5
31/31 [==============================] - 1s 24ms/step - loss: 1.6251e-14 - mae: 1.5957e-07 - val_loss: 1.9037e-14 - val_mae: 1.5308e-07
Epoch 4/5
31/31 [==============================] - 1s 22ms/step - loss: 1.6193e-14 - mae: 1.5761e-07 - val_loss: 1.9101e-14 - val_mae: 1.5230e-07
Epoch 5/5
1/1 [==============================] - 0s 100ms/step - loss: 1.9054e-14 - mae: 1.5230e-07


0.15230004635213845

#### Wavenet Implementation

In [26]:
class GatedActivationUnit(tf.keras.layers.Layer):
    def __init__(self, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)

    def call(self, inputs):
        n_filters = inputs.shape[-1] // 2
        linear_output = self.activation(inputs[..., :n_filters])
        gate = tf.keras.activations.sigmoid(inputs[..., n_filters:])
        return self.activation(linear_output) * gate

In [27]:
def wavenet_residual_block(inputs, n_filters, dilation_rate):
    z = tf.keras.layers.Conv1D(2 * n_filters, kernel_size=2, padding="causal",
                            dilation_rate=dilation_rate)(inputs)
    z = GatedActivationUnit()(z)
    z = tf.keras.layers.Conv1D(n_filters, kernel_size=1)(z)
    return tf.keras.layers.Add()([z, inputs]), z

In [28]:
tf.random.set_seed(42)

n_layers_per_block = 3  # 10 in the paper
n_blocks = 1  # 3 in the paper
n_filters = 32  # 128 in the paper
n_outputs = 14  # 256 in the paper

inputs = tf.keras.layers.Input(shape=[None, 5])
z = tf.keras.layers.Conv1D(n_filters, kernel_size=2, padding="causal")(inputs)
skip_to_last = []
for dilation_rate in [2**i for i in range(n_layers_per_block)] * n_blocks:
    z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
    skip_to_last.append(skip)

z = tf.keras.activations.relu(tf.keras.layers.Add()(skip_to_last))
z = tf.keras.layers.Conv1D(n_filters, kernel_size=1, activation="relu")(z)
Y_preds = tf.keras.layers.Conv1D(n_outputs, kernel_size=1)(z)

full_wavenet_model = tf.keras.Model(inputs=[inputs], outputs=[Y_preds])

In [29]:
fit_and_evaluate(full_wavenet_model, longer_train, longer_valid, learning_rate=0.1, epochs=5)

Epoch 1/5
31/31 [==============================] - 4s 45ms/step - loss: 8.1272e-14 - mae: 3.2156e-07 - val_loss: 2.3475e-14 - val_mae: 1.5996e-07
Epoch 2/5
31/31 [==============================] - 1s 27ms/step - loss: 1.7950e-14 - mae: 1.5280e-07 - val_loss: 1.9696e-14 - val_mae: 1.5076e-07
Epoch 3/5
31/31 [==============================] - 1s 30ms/step - loss: 1.6353e-14 - mae: 1.5501e-07 - val_loss: 1.8895e-14 - val_mae: 1.5143e-07
Epoch 4/5
31/31 [==============================] - 1s 28ms/step - loss: 1.6229e-14 - mae: 1.5427e-07 - val_loss: 1.8999e-14 - val_mae: 1.5019e-07
Epoch 5/5
1/1 [==============================] - 0s 160ms/step - loss: 1.8927e-14 - mae: 1.5038e-07


0.15037845457754884